In [31]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
import csv
import tensorflow as tf
import nltk 

import gensim as gs

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from tensorflow.contrib import rnn

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

nltk.download('punkt')

df1 = pd.read_csv('HillaryTrumpTrainWithTags.csv')
df2 = pd.read_csv('TrumpHillaryTest.csv')
"""
df2 = pd.read_table('glove.twitter.27B.25d.txt', sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

def vectorize(w):
  return df2.loc[w].as_matrix()
"""
  

[nltk_data] Downloading package punkt to /Users/Alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'\ndf2 = pd.read_table(\'glove.twitter.27B.25d.txt\', sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)\n\ndef vectorize(w):\n  return df2.loc[w].as_matrix()\n'

In [39]:
def handleToNumber(handle):
    if handle == "realDonaldTrump":
        return 0
    else:
        return 1

In [40]:
df1['tokenized_tweets'] = df1.apply(lambda row: nltk.word_tokenize(row['tweet']), axis=1)
df1['labels'] = df1.apply(lambda row: handleToNumber(row['handle']), axis=1)
df2['tokenized_tweets'] = df2.apply(lambda row: nltk.word_tokenize(row['tweet']), axis=1)

In [4]:
tweets = df1['tokenized_tweets']
tweets_test = df2['tokenized_tweets']

In [5]:
length = []
_ = df1.apply(lambda row: length.append(len(row['tokenized_tweets'])), axis=1)
max_length = max(length)

In [6]:
max_length

54

In [7]:
vectorizer = gs.models.Word2Vec(df1['tokenized_tweets'].append(df2['tokenized_tweets']) , min_count=1, size = 25, iter = 10)

In [8]:
train_matrix = []

for i, tweet in enumerate(tweets):
    #print(type(string))
    train_matrix.append(vectorizer.wv[tweet])

In [9]:
# np.array(train_matrix).reshape(len(train_matrix),1)

In [10]:
train_matrix = np.array(train_matrix)
train_matrix[0]

array([[ 6.39693737e-02, -2.80195862e-01,  1.64104879e+00,
         1.41863525e-01,  6.24521136e-01, -3.58341157e-01,
        -1.92452103e-01, -5.31973243e-01,  1.59815454e+00,
         9.93977726e-01,  2.09545031e-01, -9.64298099e-02,
         5.62113672e-02, -7.47769058e-01, -3.99524659e-01,
        -1.18690169e+00, -2.25959134e+00,  3.96619976e-01,
         3.51921469e-01, -1.05039632e+00,  1.26533878e+00,
         6.22419238e-01,  6.46917164e-01,  5.51957130e-01,
         2.45440125e-01],
       [-6.23479113e-02,  1.53254986e-01,  3.61090690e-01,
         2.39134073e-01,  3.29183251e-01, -3.03277016e-01,
         2.58549750e-01,  1.09864622e-01,  3.97847295e-01,
         6.39620185e-01,  1.21509470e-01,  9.76631120e-02,
        -1.68215886e-01, -3.64276797e-01,  1.65536717e-01,
        -4.96967822e-01, -5.93378663e-01, -1.00739367e-01,
         1.30573109e-01, -2.38590568e-01,  2.05842763e-01,
         2.17453137e-01,  4.52822834e-01,  2.17257068e-01,
         2.55629629e-01],
    

In [11]:
type(train_matrix)

numpy.ndarray

In [12]:
train_matrix.shape

(5000,)

In [13]:
train_matrix[0].shape

(23, 25)

In [14]:
twit = train_matrix[4999]

In [15]:
def pad(twits):
    """
    Does the padding as many times as required
    """
    l = len(twits)
    d= twits.shape[1]
    padding = np.zeros(d)
    
    for i in range(0,max_length-l):
        twits = np.append(twits,padding)
        
    return twits.reshape(max_length,d)

In [16]:
def pad_matrix(t_matrix):
    """
    Loops through all the vectorized tweets and calls the pad function on each of them.
    """
    
    padded_matrix =[]
    for i,vectorized_tweet in enumerate(t_matrix):
         padded_matrix.append(pad(vectorized_tweet))
    padded_matrix = np.array(padded_matrix)
    
    return padded_matrix

In [17]:
padded_train_matrix = pad_matrix(train_matrix)

In [18]:
padded_train_matrix.shape

(5000, 54, 25)

In [19]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [50]:
labels = df1['labels']
labels_vector = []

for i, label in enumerate(labels):
    labels_vector.append([label, 1 - label])
labels_vector = np.array(labels_vector)

array([[0, 1],
       [0, 1],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]])

In [48]:
#define constants
#unrolled through 28 time steps
time_steps=54
#hidden LSTM units
num_units=128
#rows of 28 pixels
n_input=25
#learning rate for adam
learning_rate=0.001
#mnist is meant to be classified in 10 classes(0-9).
n_classes=2
#size of batch
batch_size=128

#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,n_input])
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

#processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
input=tf.unstack(x ,time_steps,1)

#defining the network
lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.matmul(outputs[-1],out_weights)+out_bias

#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

#initialize variables
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    iter=1
    while iter<800:
        batch_x,batch_y=next_batch(batch_size, padded_train_matrix, labels_vector)
        sess.run(opt, feed_dict={x: batch_x, y: batch_y})

        if iter %10==0:
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
(128, 784)
(128, 10)
(5000, 54, 25)
For iter  10
Accuracy  0.8828125
Loss  0.28635377
__________________
For iter  20
Accuracy  0.84375
Loss  0.4182338
__________________
For iter  30
Accuracy  0.8359375
Loss  0.37730753
__________________
For iter  40
Accuracy  0.9453125
Loss  0.16962945
__________________
For iter  50
Accuracy  0.8671875
Loss  0.36817774
__________________
For iter  60
Accuracy  0.8984375
Loss  0.24344419
__________________
For iter  70
Accuracy  0.890625
Loss  0.30431268
__________________
For iter  80
Accuracy  0.8828125
Loss  0.2812508
__________________
For iter  90
Accuracy  0.921875
Loss  0.21496773
__________________
For iter  100
Accuracy  0.9375
Loss  0.2061697
__________________
For iter  110
Accuracy  0.890625
Loss  0.27327046
__________________
For iter  120
Accuracy  